<center>
    <h1>Colchicine Notebook</h1>
    <h3>Authors: Itai Ben-Nun, Ariel Cohen and Uri Raviv</h3>
</center>

## Supporting Libraries and functions ##

### Libraries

In [1]:
# Additional
import numpy as np
import matplotlib.pyplot as plt
import os

# D+ libraries
from dplus.CalculationInput import CalculationInput
from dplus.DataModels.models import PDB
from dplus.DataModels import ManualSymmetry, Population
from dplus.CalculationRunner import EmbeddedLocalRunner

### Global Varbs

In [2]:
pdb_col = 'Files/col_dimer.pdb'
pdb_3j6f = 'Files/3j6f_Dimer.pdb'

### Functions

#### createDOL
Creates DOL file from list/numpy matrix.

In [3]:
def createDOL(filename, DOL):
        dolfile = open(filename + '.dol','w+')
        for i in range(len(DOL)):
               dolfile.write(str(i)+'\t'+str(DOL[i][0])+'\t'+str(DOL[i][1])+'\t'+str(DOL[i][2])+'\t'+str(DOL[i][3])+'\t'+str(DOL[i][4])+'\t'+str(DOL[i][5])+'\n')      
        dolfile.close()
        print('DOL file ' + filename + ' has been created successfully!')

#### rotateDOL
Rotates DOL matrix using tait-byran rotation matrixes

In [4]:
def rotateDOL(DOL, alpha_shift=0, beta_shift=0, gamma_shift=0, filename=''):
    def A_x(angle):
        return np.array([[1, 0, 0],
                         [0, np.cos(angle), -np.sin(angle)],
                         [0, np.sin(angle), np.cos(angle)]])

    def A_y(angle):
        return np.array([[np.cos(angle), 0, np.sin(angle)],
                         [0, 1, 0],
                         [-np.sin(angle), 0, np.cos(angle)]])

    def A_z(angle):
        return np.array([[np.cos(angle), -np.sin(angle), 0],
                         [np.sin(angle), np.cos(angle), 0],
                         [0, 0, 1]])
    DOL_rotated = DOL.copy()

    alpha_shift, beta_shift, gamma_shift = np.radians([alpha_shift, beta_shift, gamma_shift])
    A = A_x(alpha_shift) @ A_y(beta_shift) @ A_z(gamma_shift)
    for i in range(len(DOL_rotated)):
        x, y, z = np.dot(A, DOL[i, :3])
        alpha, beta, gamma = np.radians(DOL[i, 3:])

        A_o = A_x(alpha) @ A_y(beta) @ A_z(gamma)  # Orientation Mat
        A_o_tag = np.dot(A, A_o)
        
        # Euler Angles
        beta_new = np.arcsin(A_o_tag[0, 2]) * 180 / np.pi
        gamma_new = np.arctan2(-A_o_tag[0, 1], A_o_tag[0, 0]) * 180 / np.pi
        alpha_new = np.arctan2(-A_o_tag[1, 2], A_o_tag[2, 2]) * 180 / np.pi

        if gamma_new < 0:
            gamma_new += 360
        if beta_new < 0:
            beta_new += 360
        if alpha_new < 0:
            alpha_new += 360

        DOL_rotated[i, :] = [x, y, z, alpha_new, beta_new, gamma_new]

    if filename:
        createDOL(filename, DOL_rotated)
    return DOL_rotated


## Models

### ITT

In [5]:
def ITT(R, L_dim, h, pitch, alpha_shift=0, beta_shift=0, gamma_shift=0, filename=''):
        p = pitch
        L_turn = np.sqrt((2*np.pi * R)**2 + p**2) 
        i_max = int((h* L_turn)/(p * L_dim) + 1) 
        theta_dim = L_dim/ R
        alpha = 90 - np.arcsin(p/L_turn) * 180/ np.pi + alpha_shift
        c = 0
        DOL = np.zeros([i_max, 6])
        for i in range(i_max):
                theta = np.remainder(i * theta_dim, np.pi * 2)
                x = R * np.cos(theta)
                y = R * np.sin(theta)
                z = i * p * L_dim / L_turn
                beta = (180 * theta / np.pi) + beta_shift
                DOL[c] = [x, y, z, alpha, beta, gamma_shift]
                c += 1
        DOL[:, 2] -= np.max(DOL[:, 2])/2
        if filename:
                createDOL(filename, DOL)
        return DOL

### CST

#### clacLen 
Calculate the length of the curve, S(t) function.

In [6]:
def calcLen(t, c, p, R):
    term1 = ((p**2/(4*np.pi**2) + c**2 + R**2) * np.arcsinh((2*c**2*t - 2*R*c) / 
            np.sqrt(4*c**2 * (p**2/(4*np.pi**2) + c**2 + R**2) - 4*R**2*c**2))) / (2*c)
    term2 = (R**2 * np.arcsinh((2*c**2*t - 2*R*c) / 
            np.sqrt(4*c**2 * (p**2/(4*np.pi**2) + c**2 + R**2) - 4*R**2*c**2))) / (2*c)
    term3 = t * np.sqrt(c**2*t**2 - 2*R*c*t + p**2/(4*np.pi**2) + c**2 + R**2) / 2
    term4 = R * np.sqrt(c**2*t**2 - 2*R*c*t + p**2/(4*np.pi**2) + c**2 + R**2) / (2*c)
    return term1 - term2 + term3 - term4

#### find_t
Binary search with convergence tolerence eps in nm.

In [7]:
def find_t(t_max, c, p, R_min, R_max,L_pitch, eps=1e-14):
        t_values = [0]
        dt = R_max/L_pitch
        L_max = calcLen(t_max,c,p,R_min) - calcLen(t_values[0],c,p,R_min) + L_pitch
        L_t = calcLen(t_values[0],c,p,R_min) + L_pitch
        while L_max - L_t > L_pitch:
                t_low = t_values[-1]
                t_high = t_low + dt
                while t_high - t_low > eps:
                        t_mid = (t_low + t_high) / 2
                        L_mid = calcLen(t_mid, c, p, R_min) - calcLen(t_values[-1], c, p, R_min)
                        if L_mid > L_pitch:
                                t_high = t_mid
                        else:
                                t_low = t_mid
                        next_t = t_low
                if next_t - t_values[-1] < eps:
                        break
                t_values.append(next_t)
                L_t = calcLen(t_values[-1],c,p,R_min) - calcLen(t_values[0],c,p,R_min) + L_pitch
        return np.array(t_values)  

#### CST
Create single CST.

In [8]:
# loops is n, defines as the number of helical turns
def CST(R_max, R_min, L_dim, pitch, loops=3, alpha_shift=0, beta_shift=0, gamma_shift=0, filename=''):
        c_value = (R_max - R_min)/(2 * np.pi * loops)
        t = find_t(2*np.pi*loops,c_value,pitch,R_min,R_max,L_dim, eps=1e-14)
        dt = np.diff(t)
        t = t[:-1]
        N_tot = len(t)
        DOL = np.zeros([N_tot, 6])
        c = 0
        for i in range(N_tot):
                x = (R_max - c_value*t[i])*np.cos(t[i])
                y = (R_max - c_value*t[i])*np.sin(t[i])
                z = pitch/(2*np.pi) * t[i]
                alpha = 90 - np.arcsin((pitch*dt[i])/(2*np.pi * L_dim)) * 180/ np.pi + alpha_shift
                beta = (180 * t[i] / np.pi) + beta_shift
                DOL[c] = [x, y, z, alpha, beta, gamma_shift]
                c += 1
        if filename:
                createDOL(filename, DOL)
        return DOL

#### Vertical Repitions 
Duplicate CST units in z-axis.

In [9]:
def VerticalRep(DOL, repetitions=1, verticalSpacing=0, filename=''):
    h = np.max(DOL[:, 2]) - np.min(DOL[:, 2])
    Vh = h + verticalSpacing
    VDOL, DOL2 = DOL.copy(), DOL.copy()
    for i in range(1, repetitions):
        DOL2[:, 2] += Vh
        VDOL = np.vstack((VDOL,DOL2))

    # Ensure center of mass
    VDOL[:, 2] -= (np.max(VDOL[:, 2]) - np.min(VDOL[:, 2]))/2
    if filename:
        createDOL(filename, VDOL)
    return VDOL

## Bundles 

### Square Bundles

In [10]:
def Square_Bundle(DOL, n1=3, n2=3, gamma_lat=90, horizontalSpacing=0, unitRad=False, antiParallel=False, filename=''):
        if n1 + n2 < 2:
                raise Exception('At least 2 layers is neccesery!')
        if n1 == 1 and n2 == 1:
               if filename:
                      createDOL(filename, DOL)
               return DOL
        if not unitRad:
                unitRad = np.max(np.sqrt(DOL[:, 0]**2 + DOL[:, 1]**2))
                
        gamma_lat -= 90        
        gamma_lat *= np.pi / 180 # Degrees to Radians
        
        a = unitRad*2 + horizontalSpacing
        a1x, a1y = a, 0
        a2x, a2y = a * np.sin(gamma_lat), a * np.cos(gamma_lat)

        n1 -= 1
        n2 -= 1
        Tx = (a1x * n1 + a2x * n2)/2
        Ty = (a1y * n1 + a2y * n2)/2
        n1, n2 = n1 + 1, n2 + 1
        
        DOL_Mirror = rotateDOL(DOL, 180)

        SDOL = np.zeros([len(DOL)*n1*n2, 6])
        c = 0
        for i in range(n1):
                for j in range(n2):
                        cx = i * a1x + j * a2x - Tx
                        cy = i * a1y + j * a2y - Ty
                        theta_shift = np.arctan2(cy, cx) * 180 / np.pi
                        DOL_i = rotateDOL(DOL,0,0,theta_shift, False)
                        DOL_Mirror_i = rotateDOL(DOL_Mirror,0,0,theta_shift, False)
                        for k in range(len(DOL)):
                                if antiParallel and (i + j) % 2 == 1:
                                        x, y, z, alpha, beta, gamma = DOL_Mirror_i[k]
                                else:
                                        x, y, z, alpha, beta, gamma = DOL_i[k]
                                x += cx
                                y += cy
                                SDOL[c] = [x, y, z, alpha, beta, gamma]
                                c += 1  

        if filename:
                createDOL(filename, SDOL)
        return SDOL    



### Hexagonal Bundles

In [11]:
def Hexagonal_Bundle(DOL, layers=3, horizontalSpacing=0, unitRad=False, filename=''):
        if layers < 1:
                raise Exception('At least 2 layers is neccesery!')

        if not unitRad:
                unitRad = np.max(np.sqrt(DOL[:, 0]**2 + DOL[:, 1]**2))
        a = unitRad*2 + horizontalSpacing

        N_IT = np.insert(np.arange(6, layers*6,6,dtype=int), 0, 1)
        HDOL = np.zeros([len(DOL)*np.sum(N_IT), 6])
        c = 0
        for IT in range(len(N_IT)):
                if IT < 2:
                        ringAngles = np.linspace(0, np.pi*2, N_IT[IT], endpoint=False) if N_IT[IT] > 0 else np.array([0])
                        cx = a * IT * np.cos(ringAngles)
                        cy = a * IT * np.sin(ringAngles)
                else:
                        ringAngles = np.linspace(0, np.pi*2, 6, endpoint=False)
                        corner_cx = a * IT * np.cos(ringAngles)
                        corner_cy = a * IT * np.sin(ringAngles)

                        cx = np.array([])
                        cy = np.array([])
                        for i in range(6):
                                x1, y1 = corner_cx[i], corner_cy[i]
                                x2, y2 = corner_cx[(i + 1) % 6], corner_cy[(i + 1) % 6]
                                for j in range(IT):
                                        frac = j / IT
                                        ring_x = x1 * (1 - frac) + x2 * frac
                                        ring_y = y1 * (1 - frac) + y2 * frac
                                        cx = np.append(cx, ring_x)
                                        cy = np.append(cy, ring_y)          
                for i in range(N_IT[IT]):
                        theta_shift = np.arctan2(cy[i], cx[i]) * 180 / np.pi
                        DOL_i = rotateDOL(DOL,0,0,theta_shift, False)
                        for j in range(len(DOL)):
                                x, y, z, alpha, beta, gamma = DOL_i[j]
                                x += cx[i]
                                y += cy[i]
                                HDOL[c] = [x, y, z, alpha, beta, gamma]
                                c += 1
        if filename:
                createDOL(filename, HDOL)
        return HDOL

## Run Models

### Hexagonal ITT Bundle runner

In [12]:
def hexagonal_ITT(folder, L_dim =8.3, horizontalSpacing=5.8, R = 22.3, h=50, pitch=0, alpha_shift=0, beta_shift=0, gamma_shift=0, layers=3,
              q_min=0, q_max=2.5, grid_size=500, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              sigma=0.01, suffix='', folderpath = 'Files'):
    

    runner = EmbeddedLocalRunner()

    # Setting name for iter_method
    if iter_method == 1:
        iter_method = 'Monte Carlo (Mersenne Twister)'
    elif iter_method == 2:
        iter_method = 'Adaptive (VEGAS) Monte Carlo'
    elif iter_method == 3:
        iter_method = 'Adaptive Gauss Kronrod'

    # Parameters for file names
    dolname = 'dol_' + suffix
    outname = 'out_' + suffix

    # Create Folder, dol, info and save parameters
    foldername = folder
    fullfolderpath = os.path.join('.', folderpath, foldername)

    # Create the directory and any necessary parent directories
    os.makedirs(fullfolderpath, exist_ok=True)

    filepath = f'./{folderpath}/{foldername}'

    if not os.path.exists(filepath):
        os.mkdir(filepath)

    DO = ITT(R=R, L_dim=L_dim, h=h, pitch=pitch, alpha_shift=alpha_shift, beta_shift=beta_shift, gamma_shift=gamma_shift)
    DO = Hexagonal_Bundle(DO, layers=layers, horizontalSpacing=horizontalSpacing, unitRad=False, filename=f'{filepath}/{dolname}')

    # Actual code
    state = CalculationInput()
    state.use_gpu = True
    state.DomainPreferences.q_max = q_max
    state.DomainPreferences.q_min = q_min
    state.DomainPreferences.grid_size= grid_size
    if signal_file:
        state.DomainPreferences.signal_file = f'./{folderpath}/{signal_file}'
    state.Domain.constant= 0
    state.Domain.scale= 1
    state.DomainPreferences.apply_resolution = True
    state.DomainPreferences.resolution_sigma = sigma
    state.DomainPreferences.orientation_method = iter_method 
    state.DomainPreferences.orientation_iterations = iterations
    state.DomainPreferences.convergence = 1e-3

    my_PDB = PDB(pdb_type) 
    my_PDB.centered = True
    my_PDB.extra_params.solvent_method.value = pdb_method
    my_PDB.extra_params.solvent_ed.value = 334
    my_PDB.extra_params.outer_solvent_ed.value = outer
    my_PDB.location_params["gamma"].value = gamma_pdb * np.pi/180
    my_PDB.location_params["beta"].value = beta_pdb * np.pi/180
    my_PDB.location_params["alpha"].value = alpha_pdb * np.pi/180

    my_sym = ManualSymmetry()
    my_sym.use_grid = True
    my_sym.read_from_dol(folderpath + '/' + foldername + '/' + dolname + '.dol')   
    my_sym.children.append(my_PDB)

    state.Domain.populations[0].add_model(my_sym)
    state.Domain.populations[0].population_size = 1 

    #state.export_all_parameters(folderpath +'/' + foldername+ '/' + statename + '.state')

    output = runner.generate(state)
    output.save_to_out_file(folderpath + '/' + foldername + '/' + outname + '.out')
    del runner, state, my_PDB, my_sym, output

### Square CST Bundle runner

In [13]:
def square_CST(folder, L_dim =8.3, pitch=5.2, T_dim=5.7, R_max = 22.3, R_min = 15, loops=3.2, 
                alpha_shift=20, beta_shift=0, gamma_shift=0, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              reps=10, verticalSpacing=-0.3, n1=3, n2=3, gamma_lat=105, horizontalSpacing = 0, square_antiParallel=True,
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, sigma=0.01, suffix='',  folderpath = 'Files'):
    

    runner = EmbeddedLocalRunner()

    # Setting name for iter_method
    if iter_method == 1:
        iter_method = 'Monte Carlo (Mersenne Twister)'
    elif iter_method == 2:
        iter_method = 'Adaptive (VEGAS) Monte Carlo'
    elif iter_method == 3:
        iter_method = 'Adaptive Gauss Kronrod'

    # Parameters for file names
    dolname = 'dol_' + suffix
    outname = 'out_' + suffix

    # Create Folder, dol, info and save parameters
    foldername = folder
    fullfolderpath = os.path.join('.', folderpath, foldername)

    # Create the directory and any necessary parent directories
    os.makedirs(fullfolderpath, exist_ok=True)

    filepath = f'./{folderpath}/{foldername}'

    if not os.path.exists(filepath):
        os.mkdir(filepath)
    
    horizontalSpacing += R_min - R_max + T_dim
    
    #pitchPerUnitSquare += R_min - R_max + T_dim
    DO = CST(R_max, R_min, L_dim, pitch=pitch, loops=loops, alpha_shift=alpha_shift, beta_shift=beta_shift, gamma_shift=gamma_shift, filename='')
    DO = VerticalRep(DO, repetitions=reps, verticalSpacing=verticalSpacing, filename='')
    DO = Square_Bundle(DO, n1=n1, n2=n2, gamma_lat=gamma_lat, horizontalSpacing=horizontalSpacing, unitRad=False, antiParallel=square_antiParallel, filename=f'{filepath}/{dolname}')

    # Actual code
    state = CalculationInput()
    state.use_gpu = True
    state.DomainPreferences.q_max = q_max
    state.DomainPreferences.q_min = q_min
    state.DomainPreferences.grid_size= grid_size
    if signal_file:
        state.DomainPreferences.signal_file = f'./{folderpath}/{signal_file}'
    state.Domain.constant= 0
    state.Domain.scale= 1
    state.DomainPreferences.apply_resolution = True
    state.DomainPreferences.resolution_sigma = sigma
    state.DomainPreferences.orientation_method = iter_method 
    state.DomainPreferences.orientation_iterations = iterations
    state.DomainPreferences.convergence = 1e-3

    my_PDB = PDB(pdb_type) 
    my_PDB.centered = True
    my_PDB.extra_params.solvent_method.value = pdb_method
    my_PDB.extra_params.solvent_ed.value = 334
    my_PDB.extra_params.outer_solvent_ed.value = outer
    my_PDB.location_params["gamma"].value = gamma_pdb * np.pi/180
    my_PDB.location_params["beta"].value = beta_pdb * np.pi/180
    my_PDB.location_params["alpha"].value = alpha_pdb * np.pi/180

    my_sym = ManualSymmetry()
    my_sym.use_grid = True
    my_sym.read_from_dol(folderpath + '/' + foldername + '/' + dolname + '.dol')   
    my_sym.children.append(my_PDB)

    state.Domain.populations[0].add_model(my_sym)
    state.Domain.populations[0].population_size = 1 

    #state.export_all_parameters(folderpath +'/' + foldername+ '/' + statename + '.state')
    output = runner.generate(state)
    output.save_to_out_file(folderpath + '/' + foldername + '/' + outname + '.out')
    del runner, state, my_PDB, my_sym, output


## Usage Examples

### CST
Modulating the structure of conical spiral tubules with colchicine.

####  $0\, \mu M$

In [14]:
square_CST(folder='Model/CST/CST_0', L_dim =8.3, pitch=5.6, T_dim=5.2, R_max = 21.5, R_min = 14.5, loops=3.4, 
                alpha_shift=20, beta_shift=0, gamma_shift=0, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              reps=4, verticalSpacing=-1.2, n1=4, n2=4, gamma_lat=105,horizontalSpacing = -1.8, square_antiParallel=True,
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_3j6f, sigma=0.01, suffix='',  folderpath = 'Files')

DOL file ./Files/Model/CST/CST_0/dol_ has been created successfully!


####  $20\, \mu M$

In [15]:
square_CST(folder='Model/CST/CST_20', L_dim =8.3, pitch=5.6, T_dim=5.2, R_max = 21.5, R_min = 14.3, loops=3.4, 
                alpha_shift=20, beta_shift=0, gamma_shift=0, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              reps=4, verticalSpacing=-1, n1=3, n2=3, gamma_lat=105,horizontalSpacing = -1.8, square_antiParallel=True,
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, sigma=0.01, suffix='',  folderpath = 'Files')


DOL file ./Files/Model/CST/CST_20/dol_ has been created successfully!


####  $40\, \mu M$ 

In [16]:
square_CST(folder='Model/CST/CST_40', L_dim =8.3, pitch=5.7, T_dim=5.2, R_max = 21.0, R_min = 13.3, loops=3.4, 
                alpha_shift=20, beta_shift=0, gamma_shift=0, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              reps=4, verticalSpacing=-1.2, n1=4, n2=4, gamma_lat=105,horizontalSpacing =-1, square_antiParallel=True,
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, sigma=0.01, suffix='',  folderpath = 'Files')


DOL file ./Files/Model/CST/CST_40/dol_ has been created successfully!


####  $80\, \mu M$ 

In [17]:
square_CST(folder='Model/CST/CST_20', L_dim =8.3, pitch=5.7, T_dim=5.2, R_max = 21, R_min = 12.8, loops=3.4, 
                alpha_shift=20, beta_shift=0, gamma_shift=0, alpha_pdb=0, beta_pdb=0, gamma_pdb=0,
              reps=6, verticalSpacing=-1.2, n1=4, n2=4, gamma_lat=105,horizontalSpacing = -1, square_antiParallel=True,
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, sigma=0.01, suffix='',  folderpath = 'Files')


DOL file ./Files/Model/CST/CST_20/dol_ has been created successfully!


### ITT
 Modulating the structure of helical tubules with colchicine.

####  $0\, \mu M$

In [18]:
# Generates single short unit
hexagonal_ITT(folder='Model/ITT/ITT_0', L_dim =9, pitch=5.2, R=20.3, h=5, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=1, horizontalSpacing = 7.5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_0M_Short', folderpath = 'Files')
    
# Generates full hexagonal structure
hexagonal_ITT(folder='Model/ITT/ITT_0', L_dim =9, pitch=5.2, R=20.3, h=99, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=4, horizontalSpacing = 7.5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_0M', folderpath = 'Files')
    


DOL file ./Files/Model/ITT/ITT_0/dol_ITT_0M_Short has been created successfully!
DOL file ./Files/Model/ITT/ITT_0/dol_ITT_0M has been created successfully!


####  $20\, \mu M$

In [19]:
# Generates single short unit
hexagonal_ITT(folder='Model/ITT/ITT_20', L_dim =9, pitch=5.2, R=19.5, h=5, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=1, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_20M_Short', folderpath = 'Files')
    
# Generates full hexagonal structure
hexagonal_ITT(folder='Model/ITT/ITT_20', L_dim =9, pitch=5.2, R=19.5, h=99, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=4, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_20M', folderpath = 'Files')



DOL file ./Files/Model/ITT/ITT_20/dol_ITT_20M_Short has been created successfully!
DOL file ./Files/Model/ITT/ITT_20/dol_ITT_20M has been created successfully!


####  $40\, \mu M$

In [20]:
# Generates single short unit
hexagonal_ITT(folder='Model/ITT/ITT_40', L_dim =9, pitch=5.2, R=18.7, h=5, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=1, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_40M_Short', folderpath = 'Files')
    
# Generates full hexagonal structure
hexagonal_ITT(folder='Model/ITT/ITT_40', L_dim =9, pitch=5.2, R=18.7, h=99, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=4, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_40M', folderpath = 'Files')


DOL file ./Files/Model/ITT/ITT_40/dol_ITT_40M_Short has been created successfully!
DOL file ./Files/Model/ITT/ITT_40/dol_ITT_40M has been created successfully!


####  $80\, \mu M$

In [21]:
# Generates single short unit
hexagonal_ITT(folder='Model/ITT/ITT_80', L_dim =9, pitch=5.2, R=18, h=5, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=1, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_80M_Short', folderpath = 'Files')
    
# Generates full hexagonal structure
hexagonal_ITT(folder='Model/ITT/ITT_80', L_dim =9, pitch=5.2, R=18, h=99, alpha_shift=0, beta_shift=0, gamma_shift=30,
              layers=4, horizontalSpacing = 5, 
              q_min=0, q_max=2.5, grid_size=250, iter_method=1, iterations=1e6, signal_file=False,
              outer = 364, pdb_method=4, pdb_type=pdb_col, alpha_pdb=22, beta_pdb=0, gamma_pdb=0,
              sigma=0.02, suffix='ITT_80M', folderpath = 'Files')



DOL file ./Files/Model/ITT/ITT_80/dol_ITT_80M_Short has been created successfully!
DOL file ./Files/Model/ITT/ITT_80/dol_ITT_80M has been created successfully!
